In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()  # Upload your kaggle.json manually here


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rugvedpednekar","key":"7b04751c25535f141cfdd35e804cbbe4"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d hghdhygf/brain-tumor-mri-image-dataset
!unzip brain-tumor-mri-image-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: 80 of training image with DAUG (5k Each Total 20k)/80% of training image with DAUG (5k Each, Total 20k)/pituitary/Tr-pi_0207_flipped_horizontal.jpg  
  inflating: 80 of training image with DAUG (5k Each Total 20k)/80% of training image with DAUG (5k Each, Total 20k)/pituitary/Tr-pi_0207_flipped_vertical.jpg  
  inflating: 80 of training image with DAUG (5k Each Total 20k)/80% of training image with DAUG (5k Each, Total 20k)/pituitary/Tr-pi_0207_rotated_180.jpg  
  inflating: 80 of training image with DAUG (5k Each Total 20k)/80% of training image with DAUG (5k Each, Total 20k)/pituitary/Tr-pi_0207_rotated_90.jpg  
  inflating: 80 of training image with DAUG (5k Each Total 20k)/80% of training image with DAUG (5k Each, Total 20k)/pituitary/Tr-pi_0208_flipped_horizontal.jpg  
  inflating: 80 of training image with DAUG (5k Each Total 20k)/80% of training image with DAUG (5k Each, Total 20k)/pituitary/Tr-pi_0208_flipped_verti

In [9]:
!ls /content

'20 of testing image with DAUG (200 Each Total 800)'   kaggle.json
'80 of training image with DAUG (5k Each Total 20k)'   sample_data
 brain-tumor-mri-image-dataset.zip


In [10]:
train_dir = '/content/80 of training image with DAUG (5k Each Total 20k)'
test_dir = '/content/20 of testing image with DAUG (200 Each Total 800)'

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Correct paths
train_dir = '/content/80 of training image with DAUG (5k Each Total 20k)'
test_dir = '/content/20 of testing image with DAUG (200 Each Total 800)'

# ImageDataGenerators
train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

# Flow from directories
train_generator = train_gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_gen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 20000 images belonging to 1 classes.
Found 800 images belonging to 1 classes.


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# Define the CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 output classes
])

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Show model summary
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 111, 111, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 54, 54, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 26, 26, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,170,372 (42.61 MB)

 Trainable params: 11,169,924 (42.61 MB)

 Non-trainable params: 448 (1.75 KB)

In [3]:
!ls

sample_data


In [6]:
!unzip brain-tumor-mri-image-dataset.zip

unzip:  cannot find or open brain-tumor-mri-image-dataset.zip, brain-tumor-mri-image-dataset.zip.zip or brain-tumor-mri-image-dataset.zip.ZIP.
